# Drug Utilization EDA
Explore the drug utilization CSV found at https://data.medicaid.gov/api/1/datastore/query/a9cfe5e9-d7d8-5b87-a7db-b45a7daf84fc/0/download

In [1]:
import pandas as pd
import os

The preferred route to get all the raw data is via the Makefile.\
The following cell is just a catch all to ensure the data is available for EDA.

In [2]:
path = 'raw_data/util/drug_utilization_2014.csv'
if not os.path.exists(path):
    import get_data
    get_data.download_drug_utiliztion(path)

In [3]:
# note, this will give a memory warning
df = pd.read_csv(path)

/tmp/ipykernel_26371/3280352130.py:1: DtypeWarning: Columns (2,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


### Efficient File Handling
Since this is such a large file, scanning each column to determine data type is expensive and gives the warning above.\
Since I've got it loaded now, list out what is being used for each column to make opening more efficient - keeping in mind the mixed types in columns 2 and 5.

In [4]:
df.dtypes

utilization_type                   object
state                              object
ndc                                object
labeler_code                        int64
product_code                        int64
package_size                       object
year                                int64
quarter                             int64
suppression_used                     bool
product_name                       object
units_reimbursed                  float64
number_of_prescriptions           float64
total_amount_reimbursed           float64
medicaid_amount_reimbursed        float64
non_medicaid_amount_reimbursed    float64
dtype: object

In [5]:
# note, due to mixed types, have to use object instead of float for columns 2 and 5
dtypes = {
    'utilization_type':'object',
    'state': 'object',
    'ndc': 'object', #'int64',
    'labeler_code': 'int64',
    'product_code': 'int64',
    'package_size': 'object', #'float64',
    'year': 'int64',
    'quarter': 'int64',
    'suppression_used': 'bool',
    'product_name': 'object',
    'units_reimbursed': 'float64',
    'number_of_prescriptions': 'float64',
    'total_amount_reimbursed': 'float64',
    'medicaid_amount_reimbursed': 'float64',
    'non_medicaid_amount_reimbursed': 'float64',
}

Confirm we no longer get a warning when loading based on specific types

In [6]:
df = pd.read_csv(path, dtype=dtypes)

Now we can load it efficiently, let's see what is actually in the file.

In [7]:
df.shape

(4315519, 15)

### Examine the first 3 rows to get a feel for the data

In [8]:
df.head(3)

,utilization_type,state,ndc,labeler_code,product_code,package_size,year,quarter,suppression_used,product_name,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
0,FFSU,AK,00002197590,2,1975,90,2014,4,True,AXIRON,NaN,NaN,NaN,NaN,NaN
1,FFSU,AK,00002322730,2,3227,30,2014,4,False,STRATTERA,942.0,23.0,7469.27,6751.02,718.25
2,FFSU,AK,00002322830,2,3228,30,2014,4,False,STRATTERA,2964.0,80.0,24272.29,22274.59,1997.70


The NaN's were unexpected.  Let's look at some table stats.

In [9]:
df.describe()

,labeler_code,product_code,year,quarter,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
count,4.315519e+06,4.315519e+06,4315519.0,4.315519e+06,2.296919e+06,2.296919e+06,2.296919e+06,2.296919e+06,2.296919e+06
mean,2.825747e+04,1.748470e+03,2014.0,2.514951e+00,3.237841e+04,5.394589e+02,4.160589e+04,4.006196e+04,1.543936e+03
std,2.839028e+04,2.342017e+03,0.0,1.118611e+00,3.507061e+05,4.650760e+03,1.222001e+06,1.215269e+06,3.325209e+04
min,2.000000e+00,-7.800000e+02,2014.0,1.000000e+00,0.000000e+00,1.100000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.780000e+02,2.110000e+02,2014.0,2.000000e+00,8.200000e+02,2.300000e+01,3.859100e+02,3.665000e+02,0.000000e+00
50%,1.671400e+04,5.930000e+02,2014.0,3.000000e+00,2.461000e+03,5.800000e+01,1.471450e+03,1.402080e+03,1.000000e-02
75%,5.974600e+04,2.586000e+03,2014.0,4.000000e+00,9.532000e+03,2.010000e+02,6.786490e+03,6.457850e+03,6.300000e+01
max,9.920700e+04,9.999000e+03,2014.0,4.000000e+00,6.005202e+07,1.423366e+06,1.061050e+09,1.061049e+09,7.812896e+06


The first thing I notice is while there are 5M rows, only about 1/2 of them have entries for: 
* units_reimbursed
* number_of_prescriptions
* total_amount_reimbursed
* medicaid_amount_reimbursed
* non_medicaid_amount_reimbursed

These seem to correlate with suppression_used as false.  To confirm, we'll perform the following two tests
1. Whenever suppression_used is true, all the above fields are na (no non na fields)
2. Whenever suppression_used is false, none of the above fields are na (no na fields)

##### 1. Whenever suppression_used is true, all the above fields are na (no non na fields)

In [10]:
# ensure count of suppression_used==True > 0
df[df['suppression_used']].shape[0]

2018600

In [11]:
# Find the intersection of suppression_used and any non-na values
# A count of 0 means no overlap between supression true and any other of specified fields having a non-na value

df[(df['suppression_used']) & 
   (
       (~df['number_of_prescriptions'].isna()) | 
       (~df['units_reimbursed'].isna()) | 
       (~df['total_amount_reimbursed'].isna()) |
       (~df['medicaid_amount_reimbursed'].isna()) |
       (~df['non_medicaid_amount_reimbursed'].isna()) 
)].shape[0]

0

##### 2. Whenever suppression_used is false, none of the above fields are na (no na fields)

In [12]:
# ensure count of suppression_used==False > 0
df[~df['suppression_used']].shape[0]

2296919

In [13]:
# Find the intersection of suppression_used==False and any na values
# A count of 0 means no overlap between supression false and any other of specified fields with value of na
df[(~df['suppression_used']) & 
   (
       (df['number_of_prescriptions'].isna()) | 
       (df['units_reimbursed'].isna()) | 
       (df['total_amount_reimbursed'].isna()) |
       (df['medicaid_amount_reimbursed'].isna()) |
       (df['non_medicaid_amount_reimbursed'].isna()) 
)].shape[0]

0

#### Conclusion
suppression_used can be used as a filter to determine if the above mentioned fields will have a non-na value

In [14]:
df = df[~df['suppression_used']].drop(columns=['suppression_used'])

### Clean NDC codes
We know that NDC codes are numeric and when they're too short, they get padded with leading zeros.
By converting to an integer, we can use the space more efficiently, make joins easier, and drop the leading zeros.

In [15]:
df['ndc'] = pd.to_numeric(df.ndc, downcast='integer') 

### Now let's look at the data for a single drug

In [16]:
df[(df.ndc==2322730)&(df.state=='AK')]

,utilization_type,state,ndc,labeler_code,product_code,package_size,year,quarter,product_name,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
1,FFSU,AK,2322730,2,3227,30,2014,4,STRATTERA,942.0,23.0,7469.27,6751.02,718.25
8588,FFSU,AK,2322730,2,3227,30,2014,3,STRATTERA,930.0,27.0,7327.43,6308.99,1018.44
16439,FFSU,AK,2322730,2,3227,30,2014,2,STRATTERA,762.0,21.0,6088.17,4760.39,1327.78
24815,FFSU,AK,2322730,2,3227,30,2014,1,STRATTERA,855.0,25.0,6880.17,5693.59,1186.58


In [17]:
# and the same drug for a different state?
df[(df.ndc==2322730)&(df.state=='NV')]

,utilization_type,state,ndc,labeler_code,product_code,package_size,year,quarter,product_name,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
2315318,FFSU,NV,2322730,2,3227,30,2014,4,STRATTERA,2294.0,49.0,19493.37,16949.16,2544.21
2315319,MCOU,NV,2322730,2,3227,30,2014,4,STRATTERA,453.0,17.0,3646.21,3646.21,0.00
2339580,FFSU,NV,2322730,2,3227,30,2014,3,STRATTERA,2450.0,57.0,20459.61,18891.27,1568.34
2362622,MCOU,NV,2322730,2,3227,30,2014,2,STRATTERA,427.0,15.0,3104.94,3104.94,0.00
2362623,FFSU,NV,2322730,2,3227,30,2014,2,STRATTERA,2368.0,51.0,18136.82,16665.23,1471.59
2386017,MCOU,NV,2322730,2,3227,30,2014,1,STRATTERA,449.0,17.0,3272.38,3272.38,0.00
2386018,FFSU,NV,2322730,2,3227,30,2014,1,STRATTERA,2209.0,48.0,16924.64,16096.91,827.73


For this study, we're not interested in differentiating between 
* product_name (we're more interested in drug class, which we'll get from another file)
* labeler_code
* product_code
* utilization_type (FFSU = Fee For Service Utilization vs MCOU - Managed Care Organization Utilization)
* state, 
* quarter the record was taken in.  

As such we can drop these fields and collapse (sum) the numeric values.

In [18]:
df = df.groupby(['ndc', 'package_size','year'], as_index=False).agg(
    units_reimbursed=pd.NamedAgg(column="units_reimbursed", aggfunc="sum"),
    number_of_prescriptions=pd.NamedAgg(column="number_of_prescriptions", aggfunc="sum"),
    total_amount_reimbursed=pd.NamedAgg(column="total_amount_reimbursed", aggfunc="sum"),
    medicaid_amount_reimbursed=pd.NamedAgg(column="medicaid_amount_reimbursed", aggfunc="sum"),
    non_medicaid_amount_reimbursed=pd.NamedAgg(column="non_medicaid_amount_reimbursed", aggfunc="sum")
)

In [19]:
df.head(3)

,ndc,package_size,year,units_reimbursed,number_of_prescriptions,total_amount_reimbursed,medicaid_amount_reimbursed,non_medicaid_amount_reimbursed
0,2143480,80,2014,90.00,23.0,21635.61,21593.61,42.00
1,2197590,90,2014,1459726.22,14828.0,5853090.72,5790959.42,62131.30
2,2300475,75,2014,2049.00,400.0,75958.59,75510.47,448.12


This gives us something we can work with.